In [ ]:
!pip install pycaret
from pycaret.classification import *
import pandas as pd
from imblearn.over_sampling import SMOTE

In [25]:
features = ['TT4', 'TT4 measured', 'T4U measured', 'T3 measured', 'FTI', 'T3', 'TSH', 'T4U', 'pregnant', 'I131 treatment']

# Abrindo o dataset
dataset = pd.read_csv('/content/hypothyroid.csv')

dataset.head()

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,...,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,binaryClass
0,41,F,f,f,f,f,f,f,f,f,...,t,125,t,1.14,t,109,f,?,SVHC,P
1,23,F,f,f,f,f,f,f,f,f,...,t,102,f,?,f,?,f,?,other,P
2,46,M,f,f,f,f,f,f,f,f,...,t,109,t,0.91,t,120,f,?,other,P
3,70,F,t,f,f,f,f,f,f,f,...,t,175,f,?,f,?,f,?,other,P
4,70,F,f,f,f,f,f,f,f,f,...,t,61,t,0.87,t,70,f,?,SVI,P


In [26]:
# Transformando dados categoricos em numericos
for index in dataset.columns.values:
  dataset[index] = dataset[index].astype('category').cat.codes.values

# Processo de limpeza do dataset (removendo linhas com dados faltantes)
for i in dataset.columns.values:
  dataset.drop(dataset[dataset[i] == '?'].index, inplace=True)
dataset = dataset.drop('TBG', axis=1)
dataset.head()

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,...,T3,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,referral source,binaryClass
0,34,1,0,0,0,0,0,0,0,0,...,27,1,28,1,72,1,10,0,1,1
1,15,1,0,0,0,0,0,0,0,0,...,22,1,3,0,146,0,234,0,4,1
2,40,2,0,0,0,0,0,0,0,0,...,69,1,10,1,48,1,22,0,4,1
3,67,1,1,0,0,0,0,0,0,0,...,20,1,83,0,146,0,234,0,4,1
4,67,1,0,0,0,0,0,0,0,0,...,12,1,201,1,44,1,199,0,3,1


In [27]:
output_label_dataset = dataset['binaryClass']
dataset = dataset[features]

In [30]:
# Balanceando os dados
sm = SMOTE(random_state=42, k_neighbors=5)
dataset_balanced, output_classe = sm.fit_resample(dataset, output_label_dataset)

dataset_b = pd.DataFrame(dataset_balanced)
classes = pd.DataFrame(output_classe)
final_dataset = pd.concat([dataset_b, classes], axis=1)

In [32]:
train_size = 0.8
session = setup(data=final_dataset, target='binaryClass', train_size=train_size, data_split_shuffle=True, normalize = True)


,Description,Value
0,Session id,239
1,Target,binaryClass
2,Target type,Binary
3,Original data shape,"(6962, 11)"
4,Transformed data shape,"(6962, 11)"
5,Transformed train set shape,"(5569, 11)"
6,Transformed test set shape,"(1393, 11)"
7,Numeric features,10
8,Preprocess,True
9,Imputation type,simple


In [33]:
best_model = compare_models()

print(best_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9776,0.9951,0.9612,0.9937,0.9772,0.9551,0.9557,0.7700
xgboost,Extreme Gradient Boosting,0.9767,0.9915,0.9655,0.9876,0.9764,0.9533,0.9536,0.4750
lightgbm,Light Gradient Boosting Machine,0.9767,0.9923,0.9666,0.9865,0.9764,0.9533,0.9536,1.2420
et,Extra Trees Classifier,0.9763,0.9958,0.9605,0.9918,0.9759,0.9526,0.9531,0.9070
gbc,Gradient Boosting Classifier,0.9695,0.9896,0.9594,0.9792,0.9692,0.9389,0.9392,0.6360
dt,Decision Tree Classifier,0.9662,0.9662,0.9580,0.9742,0.9660,0.9325,0.9327,0.1090
ada,Ada Boost Classifier,0.9634,0.9868,0.9523,0.9740,0.9630,0.9267,0.9271,0.4990
knn,K Neighbors Classifier,0.9463,0.9802,0.9070,0.9844,0.9440,0.8926,0.8955,0.3000
lda,Linear Discriminant Analysis,0.8222,0.8867,0.7842,0.8495,0.8150,0.6445,0.6470,0.1030
ridge,Ridge Classifier,0.8221,0.0000,0.7839,0.8494,0.8148,0.6441,0.6467,0.0670


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=-1, oob_score=False,
                       random_state=239, verbose=0, warm_start=False)


# Resultado

## 5 melhores classificadores: 

- Random Forest Classifier
- Extreme Gradient Boosting
- Ligth Gradient Boosting Machine
- Extra Trees Classifier
- Decision Tree Classifier